# Chapter 27: Gas Optimization

---

Gas is the lifeblood of Ethereum—every computation, storage operation, and transaction costs gas. Users pay for gas in ETH, so optimizing gas consumption directly improves user experience and reduces costs. More importantly, gas limits constrain what contracts can do; inefficient code may hit block gas limits, rendering functions unusable. This chapter delves into the art and science of gas optimization, from understanding the EVM cost model to advanced techniques like assembly and Merkle proofs. By the end, you'll be able to write lean, efficient smart contracts that save your users money and keep your protocols scalable.

---

## 27.1 Understanding Gas Costs

### 27.1.1 Gas Cost Breakdown

Every operation in the EVM has a predetermined gas cost, defined in the Ethereum yellow paper and updated via EIPs. Understanding these costs helps prioritize optimization efforts.

**Major cost categories:**

| Category | Examples | Relative Cost |
|----------|----------|---------------|
| **Arithmetic** | ADD, SUB, MUL | Very cheap (3–5 gas) |
| **Memory** | MLOAD, MSTORE | Cheap (3 gas) |
| **Storage (warm)** | SLOAD (already accessed) | Cheap (100 gas) |
| **Storage (cold)** | SLOAD (first time) | Expensive (2100 gas) |
| **Storage write** | SSTORE (zero to non-zero) | Very expensive (20000 gas) |
| **Storage write** | SSTORE (non-zero to zero) | Refund (but costs upfront) |
| **External calls** | CALL, DELEGATECALL | Expensive (700+ gas) |
| **Logs** | LOG0–LOG4 | Moderate (375 + data) |
| **Contract creation** | CREATE, CREATE2 | Very expensive (32000+ gas) |

**Gas costs are dynamic:**
- **EIP-2929** (Berlin) increased costs for first-time access to storage and addresses to discourage DoS.
- **EIP-1559** introduced base fee, but that's separate from execution gas.

### 27.1.2 Opcodes and Their Costs

A deeper look at common opcodes (based on Berlin/EIP-2929 rules):

| Opcode | Gas | Description |
|--------|-----|-------------|
| `ADD`, `SUB`, `LT`, `GT`, `EQ`, `AND`, `OR`, `XOR`, `NOT` | 3 | Arithmetic/logic |
| `MUL`, `DIV`, `SDIV`, `MOD`, `SMOD` | 5 | Multiplication/division |
| `JUMP`, `JUMPI` | 8 | Control flow |
| `SLOAD` (cold) | 2100 | First load from storage |
| `SLOAD` (warm) | 100 | Subsequent loads |
| `SSTORE` (zero to non-zero) | 20000 | Set storage from 0 to non-zero |
| `SSTORE` (non-zero to non-zero) | 5000 | Update existing storage |
| `SSTORE` (non-zero to zero) | 5000 (refund 15000) | Clearing storage |
| `BALANCE` (cold) | 2600 | Get balance of address |
| `BALANCE` (warm) | 100 | Subsequent checks |
| `CALL` (with value) | 9700 + stipend | Call with ETH transfer |
| `CALL` (without value) | 2600 | Regular call |
| `DELEGATECALL`, `STATICCALL` | 2600 | Similar to CALL |
| `RETURN` | 0 | Returning data (costs memory expansion) |
| `REVERT` | 0 | Revert (costs memory) |
| `SHA3` (KECCAK256) | 30 + 6 per word | Hashing |
| `LOG0` | 375 + 8 per byte | Event with no topics |
| `LOG1` | 375 + 8 per byte + 375 | One topic |
| `CREATE` | 32000 | Contract creation |
| `CREATE2` | 32000 | Deterministic create |
| `SELFDESTRUCT` | 5000 (refund 24000) | Self-destruct |

**Memory expansion:** When you write to memory beyond previously allocated space, gas costs increase quadratically. The cost formula: `cost = (words * words) / 512`, where `words` is the number of 32-byte chunks allocated.

**Calldata:** Reading from calldata costs 4 gas per byte (16 gas per word) for non-zero bytes, 4 gas per zero byte? Actually, EIP-2028 reduced calldata gas to 16 gas per non-zero byte and 4 gas per zero byte? Let's clarify: In EVM, `CALLDATALOAD` and `CALLDATACOPY` have costs based on memory expansion and copying; the data itself has no per-byte cost. However, transaction data has a cost: each zero byte in transaction data costs 4 gas, each non-zero byte costs 16 gas (previously 68). This incentivizes using zero bytes.

---

## 27.2 Storage Optimization

Storage is by far the most expensive resource. Optimizing how you use storage yields the biggest gas savings.

### 27.2.1 Storage Slots Explained

Ethereum storage is a key-value store with 2^256 slots, each 32 bytes. Reading or writing a slot costs gas. Contracts pay for each slot they touch.

**Slot layout:**
- State variables are assigned to slots sequentially in order of declaration.
- Multiple small variables can be packed into one slot if they fit.
- Constants and immutables do not use storage; they are embedded in the bytecode.

**Example:**
```solidity
contract StorageExample {
    uint256 a;      // slot 0
    uint128 b;      // slot 1 (starts new slot)
    uint128 c;      // also slot 1 (packs with b)
    uint8 d;        // slot 2 (starts new slot)
    uint8 e;        // also slot 2
    uint16 f;       // also slot 2
    uint256 g;      // slot 3
}
```

In this example, `b` and `c` share slot 1 (both 16 bytes). `d`, `e`, `f` share slot 2 (1+1+2 = 4 bytes, leaving 28 unused). This packing saves slots.

### 27.2.2 Variable Packing

**Guidelines:**
- Order state variables by size to maximize packing.
- Use smaller integer types where possible (but beware of overflow risks).
- Group similar types together.

**Example: Unoptimized vs Optimized**

Unoptimized:
```solidity
contract Unoptimized {
    uint128 a;
    uint256 b;   // slot 1 (starts new slot)
    uint128 c;   // slot 2
    uint8 d;     // slot 3
    uint16 e;    // slot 4
    uint256 f;   // slot 5
}
// Uses 6 storage slots
```

Optimized:
```solidity
contract Optimized {
    uint256 b;   // slot 0
    uint256 f;   // slot 1
    uint128 a;   // slot 2
    uint128 c;   // also slot 2 (packed)
    uint16 e;    // slot 3
    uint8 d;     // also slot 3 (packed)
}
// Uses 4 storage slots
```

**Packing savings:**
- Each saved slot reduces deployment cost (gas for initializing zero slots) and runtime costs (fewer SLOAD/SSTORE).

### 27.2.3 Using memory vs. storage

- **Storage** is persistent and expensive.
- **Memory** is temporary and cheap (though not free).
- **Calldata** is read-only and the cheapest.

**When to use memory:**
- For local variables that don't need to persist.
- For arrays or structs that are only used temporarily.
- To avoid multiple storage reads (copy to memory, process, then write back once).

**Example: Inefficient vs Efficient**

Inefficient (multiple storage reads):
```solidity
function sumArray() public view returns (uint) {
    uint total = 0;
    for (uint i = 0; i < arr.length; i++) {
        total += arr[i];  // SLOAD each iteration
    }
    return total;
}
```

Efficient (copy to memory):
```solidity
function sumArray() public view returns (uint) {
    uint[] memory memArr = arr;  // one SLOAD per element (still cost)
    uint total = 0;
    for (uint i = 0; i < memArr.length; i++) {
        total += memArr[i];  // MLOAD (cheap)
    }
    return total;
}
```

Actually, copying the whole array to memory still requires reading each element from storage, so it's not necessarily cheaper for reading. However, if you need to modify the array, it's better to work in memory and write back once.

**Calldata:** Use `calldata` for external function parameters to save gas (no copying to memory). Example:
```solidity
function process(uint256[] calldata data) external { // calldata
    // data[i] reads from calldata directly
}
```

---

## 27.3 Code Optimization Techniques

### 27.3.1 Short-Circuiting

Boolean operators `&&` and `||` short-circuit: they evaluate from left to right and stop as soon as the result is determined. Put cheaper conditions first.

**Example:**
```solidity
// Expensive condition first: always computes
require(balance[msg.sender] >= amount && amount > 0);

// Better: cheap condition first
require(amount > 0 && balance[msg.sender] >= amount);
```

### 27.3.2 Unchecked Arithmetic

In Solidity 0.8+, arithmetic is checked for overflow/underflow, which adds gas cost. If you know overflow is impossible, wrap in `unchecked` block.

**Example:**
```solidity
function increment(uint x) public pure returns (uint) {
    unchecked {
        return x + 1;  // saves gas
    }
}
```

**Common use cases:**
- Loops where `i < max` and max is bounded.
- Math where inputs are constrained by earlier checks.

### 27.3.3 Custom Errors vs. Revert Strings

Custom errors (Solidity 0.8.4+) are cheaper than revert strings.

**Revert string:**
```solidity
require(amount > 0, "Amount must be positive");
// costs ~200 gas + length*1 gas
```

**Custom error:**
```solidity
error AmountZero();
if (amount == 0) revert AmountZero();
// costs ~20-30 gas total
```

**Gas savings:** 80-90% on error paths. Use custom errors throughout.

### 27.3.4 Efficient Loop Patterns

Loops are a common source of high gas consumption.

**Tips:**
- Cache array length to avoid reading it each iteration.
- Use `++i` instead of `i++` (pre-increment is slightly cheaper).
- Avoid modifying storage inside loops; accumulate in memory then write once.

**Example:**
```solidity
// Inefficient
function updateArray(uint256[] memory values) public {
    for (uint i = 0; i < values.length; i++) {
        data[i] = values[i];  // SSTORE each iteration
    }
}

// Efficient: store in memory first? Not applicable; data is storage. 
// Better: if you must update storage, consider batching.

// For reading: cache length
function sum() public view returns (uint) {
    uint len = data.length;
    uint total = 0;
    for (uint i = 0; i < len; ++i) {
        total += data[i];
    }
    return total;
}
```

**Gas savings:** Pre-increment saves ~5 gas per iteration; length caching saves 1 SLOAD per iteration.

---

## 27.4 Optimization Tools

### 27.4.1 Hardhat Gas Reporter

The `hardhat-gas-reporter` plugin prints gas usage for your tests, helping identify expensive functions.

**Installation:**
```bash
npm install --save-dev hardhat-gas-reporter
```

**Configuration:**
```javascript
// hardhat.config.js
require("hardhat-gas-reporter");

module.exports = {
  gasReporter: {
    enabled: true,
    currency: 'USD',
    coinmarketcap: 'YOUR_API_KEY', // optional
    gasPrice: 20, // in gwei
  },
};
```

When you run tests, it outputs a table like:
```
|  Contract  |  Method  |  Min  |  Max  |  Avg  |
|------------|----------|-------|-------|-------|
|    Token   | transfer |  512  |  812  |  612  |
```

This highlights which functions are most expensive and where to focus optimization.

### 27.4.2 Foundry Gas Snapshots

Foundry can generate gas snapshots with `forge snapshot`.

**Usage:**
```bash
forge snapshot
```

This creates a `.gas-snapshot` file with gas usage for each test. You can compare snapshots to see how changes affect gas.

**Example output:**
```
TokenTest:testTransfer() (gas: 51234)
TokenTest:testApprove() (gas: 32100)
```

Foundry also supports `--gas-report` for a detailed breakdown.

---

## 27.5 Advanced Optimization

### 27.5.1 Assembly (Yul) for Critical Paths

For maximum optimization, you can write inline assembly (Yul). This gives direct access to EVM opcodes, bypassing Solidity's safety checks.

**When to use assembly:**
- Extremely gas-sensitive functions (e.g., core DeFi operations).
- Custom hashing or cryptography.
- Optimizing loops with many iterations.

**Example: Efficient array sum in assembly**
```solidity
function sumAssembly(uint256[] memory data) public pure returns (uint256 total) {
    assembly {
        let len := mload(data)                 // length at first word
        let ptr := add(data, 0x20)             // pointer to first element
        let end := add(ptr, mul(len, 0x20))    // end pointer
        let sum := 0
        for { } lt(ptr, end) { ptr := add(ptr, 0x20) } {
            sum := add(sum, mload(ptr))
        }
        total := sum
    }
}
```

**Risks:** Assembly bypasses safety checks; one mistake can break the contract. Use sparingly and test thoroughly.

### 27.5.2 Merkle Proofs for Large Data

When you need to store large amounts of data (e.g., whitelists, merkle airdrops), storing all entries on-chain is prohibitively expensive. Instead, store a Merkle root and verify inclusion via proofs.

**Example: Merkle airdrop**
```solidity
contract MerkleAirdrop {
    bytes32 public root;
    mapping(address => bool) public claimed;

    constructor(bytes32 _root) {
        root = _root;
    }

    function claim(bytes32[] calldata proof, uint256 amount) external {
        require(!claimed[msg.sender], "Already claimed");
        bytes32 leaf = keccak256(abi.encodePacked(msg.sender, amount));
        require(verify(proof, root, leaf), "Invalid proof");
        claimed[msg.sender] = true;
        // transfer tokens
    }

    function verify(bytes32[] memory proof, bytes32 root, bytes32 leaf) internal pure returns (bool) {
        bytes32 computedHash = leaf;
        for (uint256 i = 0; i < proof.length; i++) {
            bytes32 proofElement = proof[i];
            if (computedHash <= proofElement) {
                computedHash = keccak256(abi.encodePacked(computedHash, proofElement));
            } else {
                computedHash = keccak256(abi.encodePacked(proofElement, computedHash));
            }
        }
        return computedHash == root;
    }
}
```

This stores only one root (32 bytes) instead of thousands of addresses.

### 27.5.3 Batch Operations

Performing multiple operations in a single transaction can save gas by amortizing overhead (e.g., multiple transfers).

**Example: Batch transfer**
```solidity
function batchTransfer(address[] calldata recipients, uint256[] calldata amounts) external {
    require(recipients.length == amounts.length, "Length mismatch");
    for (uint i = 0; i < recipients.length; i++) {
        _transfer(msg.sender, recipients[i], amounts[i]);
    }
}
```

**ERC-1155** natively supports batch transfers, which is why it's more gas-efficient than multiple ERC-20 transfers for NFTs.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 27 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Gas optimization is critical for user experience and contract  │
│  viability. Understanding the EVM cost model guides efforts.   │
│                                                                 │
│  Storage is the biggest expense:                               │
│    • Pack variables tightly to use fewer slots                 │
│    • Use memory for temporary data                              │
│    • Prefer calldata for read-only parameters                   │
│                                                                 │
│  Code optimizations:                                           │
│    • Short-circuit boolean expressions                         │
│    • Use unchecked blocks for safe arithmetic                  │
│    • Replace revert strings with custom errors                 │
│    • Cache array lengths, use ++i                              │
│                                                                 │
│  Tools: Hardhat Gas Reporter, Foundry snapshots.               │
│  Advanced: assembly (sparingly), Merkle proofs, batching.     │
│                                                                 │
│  Always measure gas impact of changes; optimize only where     │
│  it matters most.                                              │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 28 – MEV (Maximal Extractable Value). We'll explore what MEV is, how it's extracted, and how to protect your users from front-running, sandwich attacks, and other forms of value extraction.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='26. smart_contract_security.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../8. advanced_development_topics/28. mev.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
